In [2]:
#text_file = "eng-ban.txt"
text_file = "ban-eng.txt"
# To open the text file we need to encode the text. Here, we use 'utf8' encoding
with open(text_file, encoding="utf8") as f:
    lines = f.read().split("\n")[:-1]
text_pairs = []
for line in lines:
    bangla, english = line.split(" >>> ")
    bangla = "[start] " + bangla + " [end]"
    text_pairs.append((english, bangla))

In [3]:
import random
for i in range(len(text_pairs)):
    print(random.choice(text_pairs));

('It is a Bangla to English translation file.', '[start] এটা বাংলা থেকে ইংরেজি অনুবাদের নথি. [end]')
('I know Bangla.', '[start] আমি বাংলা জানি. [end]')
('Sorry.', '[start] দুঃখিত. [end]')
('About seventy percentage of people are literate.', '[start] বাংলাদেশের প্রায় সত্তর ভাগ মানুষ স্বাক্ষর. [end]')
('Sorry.', '[start] দুঃখিত. [end]')
('Bangladesh is a south-asian country.', '[start] বাংলাদেশ দক্ষিন এশিয়ার একটি দেশ. [end]')
('I do not know him.', '[start] আমি উনাকে চিনি না. [end]')
('Allas! We lost the game.', '[start] হায়! আমরা খেলায় হেরেছি. [end]')
('I do not know him.', '[start] আমি উনাকে চিনি না. [end]')
('Bangladesh is a developing country.', '[start] বাংলাদেশ একটি উন্নয়নশীল দেশ. [end]')
('I know Bangla.', '[start] আমি বাংলা জানি. [end]')
('Sorry.', '[start] দুঃখিত. [end]')
('I sing song in Bangla.', '[start] আমি বাংলায় গান গাই. [end]')
('Allas! We lost the game.', '[start] হায়! আমরা খেলায় হেরেছি. [end]')
('I do not know him.', '[start] আমি উনাকে চিনি না. [end]')
('Sorry.', '[sta

In [4]:
import random
random.shuffle(text_pairs)
total_data_length = len(text_pairs)
num_val_samples = int(0.15 * total_data_length)

In [5]:
num_train_samples = (total_data_length - (2 * num_val_samples))
train_pairs = text_pairs[:num_train_samples]
val_pairs = text_pairs[num_train_samples:num_train_samples + num_val_samples]
test_pairs = text_pairs[num_train_samples + num_val_samples]

In [6]:
import tensorflow as tf
import string
import re

strip_chars = string.punctuation + "?"
strip_chars = strip_chars.replace("[", "")
strip_chars = strip_chars.replace("]", "")

In [7]:
f"[{re.escape(strip_chars)}]"

'[!"\\#\\$%\\&\'\\(\\)\\*\\+,\\-\\./:;<=>\\?@\\\\\\^_`\\{\\|\\}\\~\\?]'

In [10]:
# XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
from tensorflow.keras import layers

def custom_standardization(input_string):
    lowercase = tf.strings.lower(input_string)
    return(tf.strings.regex_replace(
        lowercase, f"[{re.escape(strip_chars)}]", ""
    ))

vocab_size = 10000
sequence_length = 20
source_vectorization = layers.TextVectorization(
    max_tokens = vocab_size,
    output_mode = "int",
    output_sequence_length = sequence_length
)
target_vectorization = layers.TextVectorization(
    max_tokens = vocab_size,
    output_mode = "int",
    output_sequence_length = sequence_length + 1,
    standardize = custom_standardization
)

In [11]:
train_english_texts = [pair[0] for pair in train_pairs]
train_bangla_texts = [pair[1] for pair in train_pairs]
source_vectorization.adapt(train_english_texts)
target_vectorization.adapt(train_bangla_texts)

In [13]:
batch_size = 64

def format_dataset(eng, ban):
    eng = source_vectorization(eng)
    ban = source_vectorization(ban)
    return({"english": eng, "bangla": ban[:, :-1]}, ban[:, 1:])

def make_dataset(pairs):
    eng_texts, ban_texts = zip(*pairs)
    eng_texts = list(eng_texts)
    ban_texts = list(ban_texts)
    dataset = tf.data.Dataset.from_tensor_slices((eng_texts, ban_texts))
    dataset = dataset.batch(batch_size)
    dataset = dataset.map(format_dataset, num_parallel_calls = 4)
    return(dataset.shuffle(2048).prefetch(16).cache())

train_ds = make_dataset(train_pairs)
val_ds = make_dataset(val_pairs)

In [20]:
print(list(train_ds.as_numpy_iterator()));

IndexError: list index out of range